Questo codice si occupa di prelevare il contenuto del file queries.json e di aggiungere per ogni risultato di ogni query il valore true/false della relvelance, grazie all'uso di un modello BERT.

Team: Data Hunters

Membri: Luca Borrelli, Rainer Cabral Ilao

In [1]:
import json
from sentence_transformers import SentenceTransformer, util
from bs4 import BeautifulSoup

c:\Users\luca-\OneDrive\Desktop\University\#MAGISTRALE\2° ANNO\1° SEMESTRE\Ingegneria dei Dati\hw3\hw3_evaluation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Carica il modello di Sentence Transformers
model = SentenceTransformer('all-mpnet-base-v2')  # Puoi sostituirlo con il modello BERT desiderato

In [ ]:

def flatten_list(nested_list):
    """Appiattisce una lista nidificata in una lista piatta."""
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))  # Ricorsione per appiattire
        else:
            flat_list.append(item)
    return flat_list

# Funzione per analizzare una tabella HTML e convertirla in testo comprensibile
def parse_html_table(html_content):
    # Se il contenuto è una lista, appiattiscilo e concatenalo
    if isinstance(html_content, list):
        html_content = "".join(flatten_list(html_content))
    
    # Controlla che sia effettivamente una stringa
    if not isinstance(html_content, str):
        raise ValueError("Il contenuto HTML deve essere una stringa.")
    
    # Parsing con BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr')
    table_text = []
    for row in rows:
        cells = row.find_all(['td', 'th'])
        table_text.append(" ".join(cell.get_text(strip=True) for cell in cells))
    return "\n".join(table_text)

# Funzione per analizzare le tabelle e rispondere alle query
def process_queries(queries_path, data_folder, model, threshold=0.5):
    with open(queries_path, 'r', encoding='utf-8') as f:
        queries = json.load(f)
    
    for query_id, query_data in queries.items():
        query = query_data["query"]
        results = query_data["result"]
        
        for result in results:
            file_path = f"{data_folder}/{result['filename']}"
            
            # Leggi il file JSON con i dati della tabella
            with open(file_path, 'r', encoding='utf-8') as file:
                file_data = json.load(file)
            
            # Estrai la tabella HTML associata a id_table
            table_id = result["id_table"]
            if table_id in file_data:
                table_html = file_data[table_id]["table"]
                table_text = parse_html_table(table_html)
                
                # Confronta la query con il testo della tabella usando il modello
                similarity = util.cos_sim(model.encode(query, convert_to_tensor=True), 
                                          model.encode(table_text, convert_to_tensor=True))
                
                # Calcolo della rilevanza come grado (tra 0 e 1)
                similarity_score = similarity.item()
                
                # Aggiungi sia la rilevanza booleana che il grado di rilevanza
                result["relevance"] = similarity_score > threshold  # True se somiglianza > soglia
                result["relevance_score"] = similarity_score  # Grado di rilevanza (0-1)
                
            else:
                result["relevance"] = False
                result["relevance_score"] = 0.0
    
    # Scrivi i risultati aggiornati nel file queries.json
    with open(queries_path, 'w', encoding='utf-8') as f:
        json.dump(queries, f, indent=2)

In [29]:
# Esempio di utilizzo
queries_path = "queries.json"  # Percorso al file queries.json
data_folder = "all_tables"           # Cartella contenente i file con le tabelle
process_queries(queries_path, data_folder,model)